<a href="https://colab.research.google.com/github/pycriador/PyCSVtoXLSX/blob/main/pyCSVtoSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Projeto de Junho

Convertendo CSV e XLSX em SQL

Primeiro, vamos importar as bibliotecas básicas

In [ ]:
!pip install sqlalchemy pandas

Iniciar com os imports necessários

In [2]:
# Import SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy.types import Integer

import pandas as pd
import zipfile
import os, re

Criar arquivo SQLite e string de conexão

In [3]:
# Criar arquivo e testar conexão
engine = create_engine('sqlite:///pesquisa.db', echo=False)
sqlite_connection = engine.connect()

Importação da classe principal:

In [8]:
class appCore():

    def files(self, path, extension):
        for file in os.listdir(path):
            if os.path.isfile(os.path.join(path, file)):
                if re.search(extension, file):
                    yield file

    def read_dir(self, path=None, extension=None):
        dir_list = []
        for path_info in path:
            dirs = self.files(path_info, extension)
            for dir_info in dirs:
                dir_list.append("%s/%s" % (path_info, dir_info))
        return dir_list

    def read(self, input=None, zipped=None, sep=None, encoding='iso-8859-1'):
        if zipped:
            with zipfile.ZipFile(input, 'r') as f:
                # ZipFile.extract(member, path=None, pwd=None)
                print('Abrindo ZIP')
                for name in f.namelist():
                    print(f'Trabalhando no arquivo: {name}')
                    filename, file_extension = os.path.splitext(name)
                    if file_extension.upper() == '.CSV':
                        print('Carregando CSV')
                        self.df_base = pd.read_csv(f.open(name), sep=sep, encoding=encoding, low_memory=False)
                        ### Se for usar um ZIP, coloque a ação para cada arquivo no ZIP
                    elif file_extension.upper() == '.XLSX':
                        print('Carregando XLSX')
                        xlsx = pd.ExcelFile(f.open(name))
                        self.df_base = pd.read_excel(xlsx)
                        ### Se for usar um ZIP, coloque a ação para cada arquivo no ZIP
        else:
            filename, file_extension = os.path.splitext(input)
            if file_extension.upper() == '.CSV':
                print('Carregando CSV')
                self.df_base = pd.read_csv(input, sep=sep, encoding=encoding, low_memory=False)
            elif file_extension.upper() == '.XLSX':
                print('Carregando XLSX')
                xlsx = pd.ExcelFile(input)
                self.df_base = pd.read_excel(xlsx)

    def write(self, dtype=None):
        sqlite_table = "Base"
        print("Criando SQLite com os dados")
        if dtype:
            # if_exists='append' - Para adicionar ou fail para dar erro
            self.df_base.to_sql(sqlite_table, sqlite_connection, if_exists='replace', dtype=dtype)
        else:
            self.df_base.to_sql(sqlite_table, sqlite_connection, if_exists='replace')


Limpando arquivos antigos e criando pastas:

In [9]:
import shutil

print('Limpando arquivos antigos')
try:
  shutil.rmtree('arquivos')
except Exception as e:
  pass
  #print(f'Erro: {e}')

print('Criando a pasta "arquivos"')
!mkdir arquivos

Limpando arquivos antigos
Criando a pasta "arquivos"


Selecione os arquivos CSV ou XLSX para upload. É possível fazer upload de ZIP com vários arquivos, porém é nessário adicionar uma função adicional. Comentários no código!

In [ ]:
from google.colab import files

print('Escolha o arquivo CSV')
ap_up = files.upload()
try:
  !mv *.csv arquivos
  !mv *.xlsx arquivos
  !mv *.zip arquivos
except Exception as e:
  print(e)

Nessa parte, você já tem o ambiente pronto, só executar o próximo passo que o script se encarregará em criar uma base SQLite com o seu arquivo.

Obs. Para determinar o tipo de informação de cada coluna, você pode escolher o **dtype**, que é o time de cada coluna. O Pandas reconhece boa parte, mas se estiver apresentando erro, será necessário configurar manualmente.

In [ ]:
app = appCore()
dir_list = app.read_dir(path=['arquivos'], extension='.csv$|.zip$|.xlsx$')

if len(dir_list) > 0:
    for readfile in dir_list:
        filename, file_extension = os.path.splitext(readfile)
        print(f'Trabalhando no arquivo {readfile}')
        if file_extension.upper() == '.ZIP':
            zipped = True
        else:
            zipped = False

        app.read(input=readfile, zipped=zipped, sep=';', encoding='iso-8859-1')
        #Exemplo de dtype
        #dtype = {'COLUNA 1': Integer()}
        dtype = {}
        app.write(dtype=dtype)

Pronto! A sua base SQlite está pronta!

Vamos testar?

In [ ]:
engine = create_engine('sqlite:///pesquisa.db', echo=False)
sqlite_connection = engine.connect()
df = pd.read_sql("SELECT * FROM Base limit 10", sqlite_connection)

print(df)